# Task5

Index_X = FSR_for_force

Index_y = force

Data = Splited by Subject

## Run result

https://wandb.ai/seokjin/FSR-prediction/groups/FSR_Trainable_2023-07-19_09-38-35/workspace?workspace=user-seokjin

## Experiment id

FSR_Trainable_2023-07-19_09-38-35

## Best metric (RMSE)

184.365

In [1]:
def define_searchspace(trial):
    model = trial.suggest_categorical('model', ['fsr_model.ANN'])
    if model == 'fsr_model.LSTM':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    elif model == 'fsr_model.CNN_LSTM':
        trial.suggest_categorical('model_args/cnn_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_categorical('model_args/lstm_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/cnn_num_layer', 1, 8)
        trial.suggest_int('model_args/lstm_num_layer', 1, 8)
    elif model == 'fsr_model.ANN':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    trial.suggest_categorical('criterion', ['torch.nn.MSELoss'])
    trial.suggest_categorical('optimizer', [
        'torch.optim.Adam',
        'torch.optim.NAdam',
        'torch.optim.Adagrad',
        'torch.optim.RAdam',
        'torch.optim.SGD',
    ])
    trial.suggest_float('optimizer_args/lr', 1e-5, 1e-1, log=True)
    trial.suggest_categorical('scaler', [ 
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])
    return {
        'index_X': 'FSR_for_force',
        'index_y': 'force',
        'data_loader': 'fsr_data.get_index_splited_by_subject'
    }

In [2]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
from fsr_trainable import FSR_Trainable
import ray.tune.search
import ray.tune.search.optuna

tuner = ray.tune.Tuner(
    trainable=ray.tune.with_resources(
        FSR_Trainable, {'cpu':2},
    ),
    tune_config=ray.tune.TuneConfig(
        num_samples=100,
        scheduler=ray.tune.schedulers.ASHAScheduler(
            max_t=100,
            grace_period=1,
            reduction_factor=2,
            brackets=1,
            metric='rmse',
            mode='min',
        ),
        search_alg=ray.tune.search.optuna.OptunaSearch(
            space=define_searchspace,
            metric='rmse',
            mode='min',
        ),
    ), 
    run_config=ray.air.RunConfig(
        callbacks=[
            ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
        ],
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='rmse',
            checkpoint_score_order='min',
            checkpoint_frequency=5,
            checkpoint_at_end=True,
        ),
    ), 
)

[I 2023-07-19 09:38:35,737] A new study created in memory with name: optuna


In [3]:
results = tuner.fit()

2023-07-19 09:38:37,781	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-07-19 09:38:39,239	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


2023-07-19 09:38:39,288	INFO wandb.py:320 -- Already logged into W&B.


Trial name,date,done,hostname,iterations_since_restore,mae,mape,node_ip,pid,rmse,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
FSR_Trainable_0166525e,2023-07-19_09-56-19,True,DESKTOP-0P789CI,2,174.928,8.70461e+07,172.26.215.93,529487,295.776,1.26319,0.599463,1.26319,1689728179,2,0166525e
FSR_Trainable_01769188,2023-07-19_09-53-43,True,DESKTOP-0P789CI,1,252.559,1.54333e+08,172.26.215.93,526416,411.261,0.64573,0.64573,0.64573,1689728023,1,01769188
FSR_Trainable_03b91ca1,2023-07-19_09-51-23,True,DESKTOP-0P789CI,2,184.06,8.76312e+07,172.26.215.93,523115,303.918,1.29487,0.577222,1.29487,1689727883,2,03b91ca1
FSR_Trainable_066e4b36,2023-07-19_09-52-31,True,DESKTOP-0P789CI,16,112.182,5.59801e+07,172.26.215.93,524056,193.138,10.8394,0.777228,10.8394,1689727951,16,066e4b36
FSR_Trainable_0bc3160d,2023-07-19_09-40-43,True,DESKTOP-0P789CI,100,166.87,1.45238e+08,172.26.215.93,511860,281.204,30.7671,0.277191,30.7671,1689727243,100,0bc3160d
FSR_Trainable_11ca36ec,2023-07-19_09-53-04,True,DESKTOP-0P789CI,1,239.569,1.23369e+08,172.26.215.93,525218,385.332,0.721361,0.721361,0.721361,1689727984,1,11ca36ec
FSR_Trainable_1593ef3c,2023-07-19_09-52-54,True,DESKTOP-0P789CI,2,178.086,7.36189e+07,172.26.215.93,524952,302.235,1.74129,0.683483,1.74129,1689727974,2,1593ef3c
FSR_Trainable_1ffc41fd,2023-07-19_09-46-49,True,DESKTOP-0P789CI,4,155.179,6.09643e+07,172.26.215.93,518895,283.144,5.46966,1.34345,5.46966,1689727609,4,1ffc41fd
FSR_Trainable_208d51c4,2023-07-19_09-39-44,True,DESKTOP-0P789CI,4,239.749,3.3003e+17,172.26.215.93,511402,431.618,1.51161,0.323457,1.51161,1689727184,4,208d51c4
FSR_Trainable_20cf1756,2023-07-19_09-54-11,True,DESKTOP-0P789CI,1,223.501,1.55078e+08,172.26.215.93,527134,372.302,0.999027,0.999027,0.999027,1689728051,1,20cf1756


(_WandbLoggingActor pid=510288) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=510288) wandb: wandb version 0.15.5 is available!  To upgrade, please run:
(_WandbLoggingActor pid=510288) wandb:  $ pip install wandb --upgrade
(_WandbLoggingActor pid=510288) wandb: Tracking run with wandb version 0.15.4
(_WandbLoggingActor pid=510288) wandb: Run data is saved locally in /home/seokj/ray_results/FSR_Trainable_2023-07-19_09-38-35/FSR_Trainable_5a2a7e89_1_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_subject,index_X=FSR_for_force,index__2023-07-19_09-38-39/wandb/run-20230719_093849-5a2a7e89
(_WandbLoggingActor pid=510288) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=510288) wandb: Syncing run FSR_Trainable_5a2a7e89
(_WandbLoggingActor pid=510288) wandb: ⭐️ View project at https://wandb.ai/seokjin/FSR-prediction
(_WandbLoggingActor pid=510288) wandb: 🚀 View run at https://wandb.ai/se